# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,pisco,-13.7000,-76.2167,21.03,83,20,4.12,PE,1715096755
1,1,adamstown,-25.0660,-130.1015,21.62,78,72,5.30,PN,1715097003
2,2,acarau,-2.8856,-40.1200,30.85,66,84,5.42,BR,1715097004
3,3,isafjordur,66.0755,-23.1240,6.02,61,75,0.00,IS,1715097004
4,4,yinchuan,38.4681,106.2731,21.39,12,97,1.56,CN,1715097004


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    'Lng', 
    'Lat', 
    size = 'Humidity',
    geo = True, 
    tiles = 'OSM',
    frame_width = 800, 
    frame_height = 500, 
    color = ['Humidity','City'],
)

#Display the map plot
map_plot_1


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_cities_df = city_data_df.loc[
    (city_data_df['Max Temp'] < 27) &
    (city_data_df['Max Temp'] > 21) &
    (city_data_df['Wind Speed'] < 4.5) &
    (city_data_df['Cloudiness'] == 0)
]
# Drop any rows with null values
filtered_cities_df = filtered_cities_df.dropna()

# Display sample data
filtered_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,5,bethel,41.3712,-73.4140,24.54,42,0,3.60,US,1715096809
24,24,khadan khak,30.7477,67.7085,23.41,26,0,2.55,PK,1715097006
57,57,albany,42.6001,-73.9662,21.76,46,0,0.45,US,1715096899
83,83,haiku-pauwela,20.9219,-156.3051,21.69,85,0,0.45,US,1715097013
179,179,shache,38.4167,77.2406,24.16,18,0,2.89,CN,1715097023
291,291,olhao,37.0286,-7.8411,25.16,54,0,3.09,PT,1715097036
307,307,beira,-19.8436,34.8389,26.05,78,0,3.60,MZ,1715097037
316,316,dondo,-19.6094,34.7431,25.73,80,0,2.82,MZ,1715097038
340,340,port saint john's,-31.6229,29.5448,25.28,58,0,4.00,ZA,1715097041
349,349,karratha,-20.7377,116.8463,25.77,70,0,2.64,AU,1715097042


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Horel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Horel Name
5,bethel,US,41.3712,-73.4140,42,
24,khadan khak,PK,30.7477,67.7085,26,
57,albany,US,42.6001,-73.9662,46,
83,haiku-pauwela,US,20.9219,-156.3051,85,
179,shache,CN,38.4167,77.2406,18,
291,olhao,PT,37.0286,-7.8411,54,
307,beira,MZ,-19.8436,34.8389,78,
316,dondo,MZ,-19.6094,34.7431,80,
340,port saint john's,ZA,-31.6229,29.5448,58,
349,karratha,AU,-20.7377,116.8463,70,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows(): 
    
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bethel - nearest hotel: Hampton Inn Danbury
khadan khak - nearest hotel: No hotel found
albany - nearest hotel: No hotel found
haiku-pauwela - nearest hotel: Inn At Mama's Fish House
shache - nearest hotel: 莎車新盛賓舘
olhao - nearest hotel: Hotel Cidade de Olhão
beira - nearest hotel: Hotel Savoy
dondo - nearest hotel: No hotel found
port saint john's - nearest hotel: Outback Inn
karratha - nearest hotel: Karratha International Hotel
tshabong - nearest hotel: No hotel found
khamis mushait - nearest hotel: No hotel found
lone grove - nearest hotel: Boarders Inn & Suites by Cobblestone Hotels
janeng - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Horel Name,Hotel Name
5,bethel,US,41.3712,-73.4140,42,,Hampton Inn Danbury
24,khadan khak,PK,30.7477,67.7085,26,,No hotel found
57,albany,US,42.6001,-73.9662,46,,No hotel found
83,haiku-pauwela,US,20.9219,-156.3051,85,,Inn At Mama's Fish House
179,shache,CN,38.4167,77.2406,18,,莎車新盛賓舘
291,olhao,PT,37.0286,-7.8411,54,,Hotel Cidade de Olhão
307,beira,MZ,-19.8436,34.8389,78,,Hotel Savoy
316,dondo,MZ,-19.6094,34.7431,80,,No hotel found
340,port saint john's,ZA,-31.6229,29.5448,58,,Outback Inn
349,karratha,AU,-20.7377,116.8463,70,,Karratha International Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    size='Humidity',
    geo = True,
    tiles='OSM',
    frame_width=600,
    frame_height=400,
    color=['City', 'Humidity'],
    hover_cols=['City','Humidity', 'Hotel Name', 'Country']
 )

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Hotel Name,Country)